- https://pub.aimind.so/reinforcement-learning-meets-large-language-models-llms-aligning-human-preferences-in-llms-88c3a3f1a3f9
- https://medium.com/@oleglatypov/a-comprehensive-guide-to-proximal-policy-optimization-ppo-in-ai-82edab5db200

In [1]:
from IPython.display import Image

- We define $r_\theta(x, y)$ as the reward model being trained, parameterized by $\theta$.
- Where you can imagine that in the above equation, x is the prompt (state) given to the LLMs agent, and y is the completion (action) given by LLM:
    - $S_t=x$
    - $a_t=y$
- LLM as the RL policy
    - action: generate tokens
    - a {prompt, completion} pair => rm => reward

In [3]:
Image(url='https://miro.medium.com/v2/resize:fit:4800/format:webp/1*WzmPShlUbdZwdBO65OtrMg.png', width=600)

Having defined the objective function (above), we can use Proximal Policy Optimization (PPO) as a reinforcement learning algorithm. The main idea of PPO is to compute the gradient of the objective function with respect to the “policy” parameter (ϕ), is shown below:

In [5]:
Image(url='https://miro.medium.com/v2/resize:fit:640/format:webp/1*vbnhEy2GFJPLfIP83LvuHQ.png', width=400)

### PPO with clip

- https://cameronrwolfe.substack.com/p/proximal-policy-optimization-ppo
- https://huggingface.co/learn/deep-rl-course/unit8/clipped-surrogate-objective

为了确保训练的稳定性并防止策略发生过大的更新，可以使用**近端策略优化（PPO）**算法。PPO 的目标函数引入了一个**截断的代理损失**：

$$
L^\text{PPO}(\theta) = \mathbb{E}_{x \sim D, y \sim \pi_\theta} \left[ \min\left( r_\theta(y \mid x) A(y, x), \text{clip}\left(r_\theta(y \mid x), 1 - \epsilon, 1 + \epsilon\right) A(y, x) \right) \right]
$$

其中：

- $ r_\theta(y \mid x) = \frac{\pi_\theta(y \mid x)}{\pi_{\theta_{\text{old}}}(y \mid x)} $ 是**概率比值**。
- $ A(y, x) = R_\phi(y) - V_{\theta_{\text{old}}}(x)$ 是**优势函数**。
- $ \epsilon $ 是控制截断范围的超参数。
- $ V_{\theta_{\text{old}}}(x) $ 是估计给定 $ x $ 的期望奖励的**值函数**。

#### 奖励函数与值函数

- 定义： 奖励函数 $R(s,a)$ 定义了在状态 $s$ 下执行动作 $a$ 后获得的即时奖励。
    - 在 RLHF 中的应用： 在大型语言模型的训练中，奖励函数通常表示为 $R(y)$，用于评估模型生成的回复 $y$ 的质量。这里，奖励模型 $R_\phi(y)$ 已通过人类偏好数据进行训练，能够为每个回复 $y$ 提供一个标量奖励。
- 值函数 $V(s)$ 表示在状态 $s$ 下，遵循策略 $\pi$ 所能获得的预期累积奖励。公式为

$$
V^{\pi}(s)=\mathbb E_\pi\left[\sum_{t=0}\gamma^tR(s_t,a_t)|s_0=s\right]
$$

- 在 RLHF 中的应用： 值函数 $V(x)$ 估计了在给定提示 $x$ 下，模型按照当前策略 $\pi_\theta$ 所能获得的预期累积奖励。
-  在策略梯度方法中，策略的更新方向取决于优势函数

    $$
    A(y,x)=R(y)-V(x)
    $$
    - 优势函数衡量了特定动作（或回复） 𝑦 相对于状态 𝑥 下平均水平的优劣。